In [ ]:
import requests
import json
from datetime import datetime
import re

resultArray = []
error_urls = []
# 将获取目录的cookie字符串复制到此
str_cookie_main =\
"mstuid=xxxx; Hm_lvt_c3e3e8b3ea48955284516b186acf0f4e=xxx; uLocale=zh_CN; iplocale=zh_CN; userId=1485538246; i.mi.com_istrudev=true; i.mi.com_ph=TgLtbZZsf/60kMWpxzpldg==; i.mi.com_isvalid_servicetoken=true; serviceToken=jNa2f6UlbQ0iiMIayebNzXjfA/5lKIM4krT0g9uhqh80opAjtN38QN8Aj+oPd1qf52EDCl8i/VgpTKChJKYOs1CttaGSSTQJk8CaasyFPyzTNwP94r9cj9KVMDAyYalrXpOC3uQml4AN+xx/P2eMbbSKt/xxx+pQvHQsf8VJFIS4qpF8M0WxG7nh/xx+xx+qmc/xxx==; i.mi.com_slh=xxx="
# 将获取单条笔记的cookie字符串复制到此
str_cookie_every_note = \
"mstuid=xxxx; Hm_lvt_c3e3e8b3ea48955284516b186acf0f4e=xxx; uLocale=zh_CN; iplocale=zh_CN; userId=1485538246; i.mi.com_istrudev=true; i.mi.com_ph=TgLtbZZsf/60kMWpxzpldg==; i.mi.com_isvalid_servicetoken=true; serviceToken=jNa2f6UlbQ0iiMIayebNzXjfA/5lKIM4krT0g9uhqh80opAjtN38QN8Aj+oPd1qf52EDCl8i/VgpTKChJKYOs1CttaGSSTQJk8CaasyFPyzTNwP94r9cj9KVMDAyYalrXpOC3uQml4AN+P4c0KVSKNYqv06SdYQBt5rCIki2hJeRFs9k59PtO/xx/xx+xx/xxx+xxxxx==; i.mi.com_slh=xxx+xxx+A="
# 将cookie字符串转为字典
cookie_main = get_cookie(str_cookie_main)
cookie_dict_every_note = get_cookie(str_cookie_every_note)

# 排序指标，可以修改
def sortRule(obj):
    return obj['create_date']

# 将cookie字符串转为字典格式
def get_cookie(cookie_str):
    # 将cookie字符串转换为字典
    cookie_dict = {}
    for item in cookie_str.split('; '):
        key, value = item.split('=',1)
        cookie_dict[key] = value
    return cookie_dict

def nextPage(syncTag=None):
    url = 'https://i.mi.com/note/full/page/?limit=200'
    if syncTag:
        url += '&syncTag=' + syncTag
    try:
        response = requests.get(url,cookies=cookie_main)
    except:
        print("connection error")
    print("1: ")
    if response.status_code == 200:
        result = response.json()
        for entry in result['data']['entries']:
            print("2: ")
            detailUrl = 'https://i.mi.com/note/note/' + entry['id']
            try:
                detailResponse = requests.get(detailUrl,cookies=cookie_dict_every_note)
            except:
                print(f"error: {detailUrl} ")
                error_urls.append(detailUrl)
                continue
                
            if detailResponse.status_code == 200:
                detailInfo = detailResponse.json()['data']['entry']
                date = datetime.fromtimestamp(entry['createDate'] / 1000)
                modifyDate = datetime.fromtimestamp(entry['modifyDate'] / 1000)
                resultObj = {}
                resultObj['title'] = json.loads(detailInfo.get('extraInfo', '{}')).get('title', '无')
                resultObj['create_date'] = date
                resultObj['modify_date'] = modifyDate
                resultObj['content'] = detailInfo['content']
                resultObj['folderId'] = detailInfo['folderId']
                resultObj['colorId'] = detailInfo['colorId']
                print(detailUrl, '内容：',detailInfo['content'][:20])
                resultArray.append(resultObj)
        if result['data']['entries']:
            nextPage(result['data']['syncTag'])
        
#             resultArray.sort(key=sortRule)
#             with open('output.txt', 'w', encoding='utf-8') as f:
#                 for resObj in resultArray:
#                     f.write('创建日期：' + dateFormat(resObj['create_date']) + '\n')
#                     f.write('修改日期：' + dateFormat(resObj['modify_date']) + '\n')
#                     f.write('标题：' + resObj['title'] + '\n')
#                     f.write('内容：' + remove_tags(resObj['content']) + '\n')
#                     f.write('-----------------------\n')
#                 print("内容已保存到 output.txt 文件中")

def dateFormat(date):
    return date.strftime('%Y-%m-%d %H:%M:%S')

def remove_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

nextPage()
# 将笔记写入output.md文件中
with open('output.md', 'w', encoding='utf-8') as f:
    for resObj in resultArray:
        f.write('创建日期：' + dateFormat(resObj['create_date']) + '\n')
        f.write('修改日期：' + dateFormat(resObj['modify_date']) + '\n')
        f.write('标题：' + resObj['title'] + '\n')
        f.write('内容：' + remove_tags(resObj['content']) + '\n')
        f.write('-----------------------\n')
    print("内容已保存到 output.md 文件中")
# 将未成功的笔记链接保存到error_urls.txt文件
with open('error_urls.txt', 'w', encoding='utf-8') as f:
    for url in error_urls:
        f.write(url+'\n')
    print("error 内容已保存到文件error_urls.txt中")

In [70]:
with open('output.txt', 'w', encoding='utf-8') as f:
    for resObj in resultArray:
        f.write('创建日期：' + dateFormat(resObj['create_date']) + '\n')
        f.write('修改日期：' + dateFormat(resObj['modify_date']) + '\n')
        f.write('标题：' + resObj['title'] + '\n')
        f.write('内容：' + remove_tags(resObj['content']) + '\n')
        f.write('-----------------------\n')
    print("内容已保存到 output.txt 文件中")
with open('error_urls.txt', 'w', encoding='utf-8') as f:
    for url in error_urls:
        f.write(url+'\n')
    print("error 内容已保存到文件error_urls.txt中")

内容已保存到 output.txt 文件中
error 内容已保存到文件error_urls.txt中
